# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Latitude,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,2.50,81,78,3.80,GS,1744945387
1,1,dexter,36.7959,-89.9579,21.85,58,83,7.49,US,1744945388
2,2,wielun,51.2210,18.5696,16.75,70,76,1.24,PL,1744945389
3,3,longyearbyen,78.2186,15.6401,-10.09,79,75,2.06,SJ,1744945390
4,4,port-aux-francais,-49.3500,70.2167,5.11,64,24,12.07,TF,1744945391


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Create a map plot for the city_data_df DataFrame, enabling a geogrpahic plot with color based on city names
city_map = city_data_df.hvplot.points(
    "Lng",                     
    "Latitude",                     
    geo=True,                  
    tiles="CartoLight",        
    size="Humidity",           
    color="City",              
    alpha=0.8,                 
    frame_width=700,           
    frame_height=500           
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Latitude]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[
    (city_data_df["Max Temp"] > 21) & 
    (city_data_df["Max Temp"] < 27) & 
    (city_data_df["Wind Speed"] < 4.5) & 
    (city_data_df["Cloudiness"] == 0)
].copy()


# Drop any rows with null values and display info about remaining cities
ideal_cities_df = ideal_cities_df.dropna()
print(f"Found {len(ideal_cities_df)} cities with ideal weather conditions")
ideal_cities_df

Found 11 cities with ideal weather conditions


,City_ID,City,Latitude,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
64,64,hadibu,12.6500,54.0333,26.30,66,0,1.45,YE,1744945459
87,87,saint-pierre,-21.3393,55.4781,24.82,69,0,2.57,RE,1744945486
106,106,ulladulla,-35.3500,150.4667,25.44,55,0,3.13,AU,1744945510
125,125,bastrop,32.7562,-91.8724,23.99,60,0,3.60,US,1744945532
263,263,namibe,-15.1961,12.1522,21.92,79,0,3.14,AO,1744945692
321,321,dubbo,-32.2500,148.6167,25.07,38,0,1.54,AU,1744945761
428,428,qina,26.1642,32.7267,21.29,21,0,1.66,EG,1744945888
432,432,ares,-6.1944,-35.1603,25.42,86,0,3.09,BR,1744945892
484,484,bandrele,-12.9067,45.1914,24.86,94,0,2.06,YT,1744945955
487,487,jacmel,18.2342,-72.5347,24.47,81,0,2.88,HT,1744945958


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City", "Country", "Latitude", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df


,City,Country,Latitude,Lng,Humidity,Hotel Name
64,hadibu,YE,12.6500,54.0333,66,
87,saint-pierre,RE,-21.3393,55.4781,69,
106,ulladulla,AU,-35.3500,150.4667,55,
125,bastrop,US,32.7562,-91.8724,60,
263,namibe,AO,-15.1961,12.1522,79,
321,dubbo,AU,-32.2500,148.6167,38,
428,qina,EG,26.1642,32.7267,21,
432,ares,BR,-6.1944,-35.1603,86,
484,bandrele,YT,-12.9067,45.1914,94,
487,jacmel,HT,18.2342,-72.5347,81,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:


# Set parameters to search for a hotel
radius = 10000  # 10 km search radius
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Create a new column for storing hotel names
hotel_df["Hotel Name"] = ""

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    latitude = row["Latitude"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
hadibu - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
ulladulla - nearest hotel: Hotel Marlin
bastrop - nearest hotel: No hotel found
namibe - nearest hotel: Nelsal Pensao
dubbo - nearest hotel: Blue Gum Motel
qina - nearest hotel: Basma Hotel
ares - nearest hotel: No hotel found
bandrele - nearest hotel: Les Baobabs
jacmel - nearest hotel: Hotel L'Amenitie
zouerate - nearest hotel: فندق تيرس زمور


,City,Country,Latitude,Lng,Humidity,Hotel Name
64,hadibu,YE,12.6500,54.0333,66,No hotel found
87,saint-pierre,RE,-21.3393,55.4781,69,Tropic Hotel
106,ulladulla,AU,-35.3500,150.4667,55,Hotel Marlin
125,bastrop,US,32.7562,-91.8724,60,No hotel found
263,namibe,AO,-15.1961,12.1522,79,Nelsal Pensao


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    x="Lng",
    y="Latitude",
    geo=True,
    tiles="CartoLight",
    size="Humidity",
    color="City",
    alpha=0.8,
    hover_cols=["City", "Hotel Name", "Humidity"],
    frame_width=700,
    frame_height=500
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Latitude]   (City,Humidity,Hotel Name)